# Preprossessing & Machine Learning 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb # pip install xgboost
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from statsmodels.tsa.arima.model import ARIMA # Pour ARIMA
from prophet import Prophet # pip install prophet

import matplotlib.pyplot as plt
import seaborn as sns

# Pour la reproductibilité
np.random.seed(42)

In [2]:
#load data 
df_global=pd.read_csv("../code/achat_prod_fournisseur_stock.csv") #upload from code file

In [3]:
df_global.describe()
df_global["date_achat"]=pd.to_datetime(df_global["date_achat"])
df_global.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id_achat               10000 non-null  object        
 1   date_achat             10000 non-null  datetime64[ns]
 2   id_produit             10000 non-null  object        
 3   quantité               10000 non-null  int64         
 4   id_fournisseur         10000 non-null  object        
 5   prix_unitaire          10000 non-null  float64       
 6   délai_livraison_jours  10000 non-null  int64         
 7   montant_total          10000 non-null  float64       
 8   mois                   10000 non-null  int64         
 9   année                  10000 non-null  int64         
 10  jour_semaine           10000 non-null  int64         
 11  catégorie              10000 non-null  object        
 12  marque                 10000 non-null  object        
 13  pr

## 1

In [4]:
df_global[["niveau_stock","quantité","stock_minimum"]].describe()

,niveau_stock,quantité,stock_minimum
count,10000.000000,10000.000000,10000.000000
mean,147.771300,99.512500,53.283000
std,88.354962,98.580534,25.376137
min,0.000000,10.000000,10.000000
25%,72.000000,25.000000,31.000000
50%,145.000000,75.000000,54.000000
75%,226.000000,100.000000,74.000000
max,299.000000,500.000000,99.000000


In [5]:
# Génération de la variable cible 'niveau_stock' (catégorielle)
bins = [0, 1.5, 3, float('inf')]
labels = ['bas', 'moyen', 'élevé']
# Ratio stock actuel (quantité) par rapport au stock minimum
# (simplification, dans la réalité le stock actuel serait une autre colonne)
df_global['ratio_stock'] = df_global['niveau_stock'] / df_global['stock_minimum']
df_global['niveau_stock_cat'] = pd.cut(df_global['ratio_stock'], bins=bins, labels=labels, right=False)
# Génération de la variable cible 'rupture' (booléen)
# Si niveau_stock est 'bas' et délai de livraison élevé -> risque de rupture
df_global['rupture'] = ((df_global['niveau_stock_cat'] == 'bas') & (df_global['délai_livraison_jours'] > 15))

***explication des modifications apportées***
- Changement des bornes des catégories de niveau de stock pour mieux refléter la distribution des données.
- Activation de la création de la colonne 'niveau_stock_cat' pour catégoriser le niveau de stock.
- Activation de la création de la colonne 'rupture' pour indiquer les ruptures de stock potentielles.
    

In [6]:
# Pour la segmentation fournisseur, ajoutons 'taille_fournisseur'
taille_fourn = df_global.groupby('id_fournisseur')['id_produit'].nunique().rename('taille_fournisseur_nb_produits')
df_global = df_global.merge(taille_fourn, on='id_fournisseur', how='left')

In [7]:
# Pour la classification de fiabilité fournisseur, créons une catégorie
# Si fiabilité (score continu) > 0.85 -> fiable, sinon -> moins_fiable
df_global['fiabilité_catégorie'] = pd.cut(df_global['fiabilité'], bins=[0, 0.85, 1.01], labels=['moins_fiable', 'fiable'], right=False)

In [8]:
# --- Fonctions Utilitaires (pour éviter la répétition) ---
def get_preprocessor(numerical_features, categorical_features):
    """Crée un ColumnTransformer pour le prétraitement."""
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ], remainder='passthrough' # pour les colonnes non spécifiées (si besoin)
    )
    return preprocessor


In [9]:
# --- Tâche 1: Régression du montant total ---
print("\n--- Tâche 1: Régression du montant_total ---")
df_task1 = df_global.copy()
X1_cols_num = ['quantité', 'prix_unitaire', 'délai_livraison_jours']
X1_cols_cat = ['id_produit', 'catégorie', 'marque', 'id_fournisseur', 'ville', 'pays']
y1_col = 'montant_total'


--- Tâche 1: Régression du montant_total ---


In [10]:
X1 = df_task1[X1_cols_num + X1_cols_cat]
y1 = df_task1[y1_col]

# Séparation des données
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

# Prétraitement
preprocessor1 = get_preprocessor(X1_cols_num, X1_cols_cat)


In [11]:
# Modèles
models1 = {
    "Régression Linéaire": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
    "XGBoost Regressor": xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
}


In [12]:
for name, model in models1.items():
    print(f"\nEntraînement du modèle: {name}")
    pipeline = Pipeline(steps=[('preprocessor', preprocessor1), ('regressor', model)])
    pipeline.fit(X1_train, y1_train)
    y1_pred = pipeline.predict(X1_test)
    
    mse = mean_squared_error(y1_test, y1_pred)
    r2 = r2_score(y1_test, y1_pred)
    print(f"Résultats pour {name}:")
    print(f"  Mean Squared Error: {mse:.2f}")
    print(f"  R² Score: {r2:.2f}")



Entraînement du modèle: Régression Linéaire


Résultats pour Régression Linéaire:
  Mean Squared Error: 82158977.18
  R² Score: 0.79

Entraînement du modèle: Random Forest Regressor
Résultats pour Random Forest Regressor:
  Mean Squared Error: 14347.04
  R² Score: 1.00

Entraînement du modèle: XGBoost Regressor
Résultats pour XGBoost Regressor:
  Mean Squared Error: 158505.45
  R² Score: 1.00


In [13]:
# --- Tâche 2: Régression du délai de livraison ---
print("\n--- Tâche 2: Régression du délai_livraison_jours ---")
df_task2 = df_global.copy()
X2_cols_num = ['fiabilité', 'délai_moyen_jours', 'quantité', 'stock_minimum']
X2_cols_cat = ['id_fournisseur', 'ville', 'pays', 'entrepot']
y2_col = 'délai_livraison_jours'

X2 = df_task2[X2_cols_num + X2_cols_cat]
y2 = df_task2[y2_col]

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)
preprocessor2 = get_preprocessor(X2_cols_num, X2_cols_cat)

# Utilisons Random Forest pour cet exemple
model2 = RandomForestRegressor(random_state=42)
pipeline2 = Pipeline(steps=[('preprocessor', preprocessor2), ('regressor', model2)])
pipeline2.fit(X2_train, y2_train)
y2_pred = pipeline2.predict(X2_test)

mse2 = mean_squared_error(y2_test, y2_pred)
r2_2 = r2_score(y2_test, y2_pred)
print(f"Résultats pour Random Forest Regressor (Délai livraison):")
print(f"  Mean Squared Error: {mse2:.2f}")
print(f"  R² Score: {r2_2:.2f}")



--- Tâche 2: Régression du délai_livraison_jours ---
Résultats pour Random Forest Regressor (Délai livraison):
  Mean Squared Error: 17.22
  R² Score: -0.22


In [15]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id_achat                        10000 non-null  object        
 1   date_achat                      10000 non-null  datetime64[ns]
 2   id_produit                      10000 non-null  object        
 3   quantité                        10000 non-null  int64         
 4   id_fournisseur                  10000 non-null  object        
 5   prix_unitaire                   10000 non-null  float64       
 6   délai_livraison_jours           10000 non-null  int64         
 7   montant_total                   10000 non-null  float64       
 8   mois                            10000 non-null  int64         
 9   année                           10000 non-null  int64         
 10  jour_semaine                    10000 non-null  int64         
 11  cat

In [ ]:

# --- Tâche 3: Classification du niveau de stock ---
print("\n--- Tâche 3: Classification du niveau_stock ---")
df_task3 = df_global.copy().dropna(subset=['niveau_stock_cat']) # S'assurer qu'il n'y a pas de NaN dans la cible
X3_cols_num = ['quantité', 'stock_minimum', 'délai_livraison_jours', 'délai_moyen_jours']
X3_cols_cat = ['catégorie', 'marque', 'entrepot']
y3_col = 'niveau_stock_cat'

X3 = df_task3[X3_cols_num + X3_cols_cat]
y3 = df_task3[y3_col]

# Encodage de la variable cible si elle est textuelle
le = LabelEncoder()
y3_encoded = le.fit_transform(y3)

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3_encoded, test_size=0.2, random_state=42, stratify=y3_encoded)
preprocessor3 = get_preprocessor(X3_cols_num, X3_cols_cat)

models3 = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest Classifier": RandomForestClassifier(random_state=42),
    "SVM Classifier": SVC(random_state=42)
}

for name, model in models3.items():
    print(f"\nEntraînement du modèle: {name}")
    pipeline = Pipeline(steps=[('preprocessor', preprocessor3), ('classifier', model)])
    pipeline.fit(X3_train, y3_train)
    y3_pred = pipeline.predict(X3_test)
    
    accuracy = accuracy_score(y3_test, y3_pred)
    print(f"Résultats pour {name}:")
    print(f"  Accuracy: {accuracy:.2f}")
    print("  Classification Report:")
    # Afficher les noms des classes au lieu des entiers encodés
    print(classification_report(y3_test, y3_pred, target_names=le.classes_, zero_division=0))




--- Tâche 3: Classification du niveau_stock ---

Entraînement du modèle: Logistic Regression
Résultats pour Logistic Regression:
  Accuracy: 0.27
  Classification Report:


TypeError: object of type 'numpy.int64' has no len()

In [22]:
# --- Tâche 4: Prédiction de rupture de stock ---
print("\n--- Tâche 4: Prédiction de rupture_de_stock ---")
df_task4 = df_global.copy()
# Ajout des features temporelles si 'date_achat' est disponible
df_task4['mois'] = df_task4['date_achat'].dt.month
df_task4['année'] = df_task4['date_achat'].dt.year
df_task4['jour_semaine'] = df_task4['date_achat'].dt.dayofweek

X4_cols_num = ['stock_minimum', 'quantité', 'délai_moyen_jours', 'mois', 'année', 'jour_semaine']
X4_cols_cat = ['niveau_stock', 'entrepot'] # niveau_stock est déjà catégoriel
y4_col = 'rupture' # booléen (0 ou 1)

X4 = df_task4[X4_cols_num + X4_cols_cat]
y4 = df_task4[y4_col].astype(int) # Conversion booléen vers int

X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.2, random_state=42, stratify=y4)
preprocessor4 = get_preprocessor(X4_cols_num, X4_cols_cat)

model4 = RandomForestClassifier(random_state=42)
pipeline4 = Pipeline(steps=[('preprocessor', preprocessor4), ('classifier', model4)])
pipeline4.fit(X4_train, y4_train)
y4_pred = pipeline4.predict(X4_test)

accuracy4 = accuracy_score(y4_test, y4_pred)
print(f"Résultats pour Random Forest Classifier (Rupture de stock):")
print(f"  Accuracy: {accuracy4:.2f}")
print("  Classification Report:")
print(classification_report(y4_test, y4_pred, zero_division=0))
print("  Confusion Matrix:")
print(confusion_matrix(y4_test, y4_pred))




--- Tâche 4: Prédiction de rupture_de_stock ---


KeyError: 'rupture'

In [16]:

# --- Tâche 5: Prévision de la demande (séries temporelles) ---
print("\n--- Tâche 5: Prévision de la demande (quantité par produit) ---")
# Exemple avec UN SEUL produit pour simplifier. En pratique, on ferait une boucle ou un modèle par produit.
df_task5_prodA = df_global[df_global['id_produit'] == 'PROD_A'].copy()
df_task5_prodA = df_task5_prodA.set_index('date_achat')
# Agréger par mois pour avoir une série plus lisse (optionnel)
ts_data = df_task5_prodA['quantité'].resample('M').sum()

if len(ts_data) > 12 : # Besoin d'assez de données pour Prophet/ARIMA
    # Séparation Temporelle
    train_size = int(len(ts_data) * 0.8)
    ts_train, ts_test = ts_data[0:train_size], ts_data[train_size:]

    # Modèle Prophet (souvent plus simple à mettre en oeuvre initialement)
    print("\nModèle Prophet:")
    df_prophet_train = ts_train.reset_index()
    df_prophet_train.columns = ['ds', 'y'] # Prophet requiert ces noms de colonnes

    model_prophet = Prophet()
    model_prophet.fit(df_prophet_train)

    future_dates = model_prophet.make_future_dataframe(periods=len(ts_test), freq='M')
    forecast_prophet = model_prophet.predict(future_dates)
    
    y5_pred_prophet = forecast_prophet['yhat'][-len(ts_test):].values # Prendre les prédictions pour la période de test

    # Évaluation Prophet
    # Note: L'évaluation directe sur ts_test peut nécessiter un alignement d'index
    if len(y5_pred_prophet) == len(ts_test):
        mse_prophet = mean_squared_error(ts_test.values, y5_pred_prophet)
        print(f"  MSE Prophet: {mse_prophet:.2f}")

        plt.figure(figsize=(12, 6))
        plt.plot(ts_train.index, ts_train.values, label='Train')
        plt.plot(ts_test.index, ts_test.values, label='Test Réel')
        plt.plot(ts_test.index, y5_pred_prophet, label='Forecast Prophet')
        plt.title('Prévision de la demande (PROD_A) avec Prophet')
        plt.legend()
        # plt.show() # Décommenter pour afficher le graphique
        print("Graphique Prophet généré (décommentez plt.show() pour l'afficher).")
    else:
        print("  Problème de dimension pour l'évaluation Prophet.")
        
    # Modèle ARIMA (exemple basique)
    # L'analyse ARIMA nécessite des étapes de vérification de stationnarité, détermination des ordres p,d,q (ACF/PACF)
    # Ce qui est hors de portée d'un script automatisé simple.
    # Voici un exemple avec des ordres arbitraires (à ajuster après analyse)
    print("\nModèle ARIMA (exemple basique):")
    try:
        model_arima = ARIMA(ts_train, order=(5,1,0)) # Ordres (p,d,q) à déterminer
        model_arima_fit = model_arima.fit()
        y5_pred_arima = model_arima_fit.forecast(steps=len(ts_test))

        mse_arima = mean_squared_error(ts_test.values, y5_pred_arima)
        print(f"  MSE ARIMA: {mse_arima:.2f}")
    except Exception as e:
        print(f"  Erreur ARIMA: {e}")
else:
    print("Pas assez de données pour la série temporelle du produit PROD_A.")


# --- Tâche 6: Segmentation produits ---
print("\n--- Tâche 6: Segmentation produits ---")
df_task6 = df_global.copy()
# Utiliser des moyennes par produit ou des caractéristiques statiques
# Ici, nous allons prendre la moyenne des caractéristiques numériques par produit
# et les caractéristiques catégorielles les plus fréquentes (ou les considérer comme des features globales du produit)
# Pour simplifier, prenons quelques caractéristiques et faisons une segmentation sur les produits uniques.
# Pour une vraie segmentation, on prendrait des caractéristiques agrégées par produit
# Ex: prix moyen, catégorie principale, marque principale, stock_minimum moyen, etc.
prod_features = df_task6.groupby('id_produit').agg(
    prix_moyen=('prix_unitaire', 'mean'),
    stock_min_moyen=('stock_minimum', 'mean'),
    delai_liv_moyen=('délai_livraison_jours', 'mean'),
    catégorie_mode=('catégorie', lambda x: x.mode()[0] if not x.mode().empty else 'N/A'), # Mode pour la catégorie
    marque_mode=('marque', lambda x: x.mode()[0] if not x.mode().empty else 'N/A') # Mode pour la marque
).reset_index()


X6_cols_num = ['prix_moyen', 'stock_min_moyen', 'delai_liv_moyen']
X6_cols_cat = ['catégorie_mode', 'marque_mode'] # ces colonnes sont déjà au niveau produit
X6_processed = prod_features.copy() # Pour garder une trace des id_produit

# Prétraitement
# Les colonnes catégorielles sont déjà 'agrégées', on peut les one-hot encoder directement
# Création du preprocessor
preprocessor6 = get_preprocessor(X6_cols_num, X6_cols_cat)
X6_prepared = preprocessor6.fit_transform(X6_processed[X6_cols_num + X6_cols_cat])

if X6_prepared.shape[0] > 1 : # K-means a besoin d'au moins n_clusters échantillons
    # K-Means
    # Déterminer le nombre optimal de clusters (méthode du coude - non implémentée ici pour brièveté)
    n_clusters_prod = min(3, X6_prepared.shape[0]) # Exemple: 3 clusters ou moins si peu de produits
    if n_clusters_prod > 1:
        kmeans_prod = KMeans(n_clusters=n_clusters_prod, random_state=42, n_init='auto')
        prod_features['cluster_kmeans'] = kmeans_prod.fit_predict(X6_prepared)
        
        silhouette_kmeans = silhouette_score(X6_prepared, prod_features['cluster_kmeans'])
        print(f"K-Means (Produits) Silhouette Score: {silhouette_kmeans:.2f}")
        print("Premiers produits avec leurs clusters K-Means:")
        print(prod_features[['id_produit', 'cluster_kmeans']].head())

        # DBSCAN (plus robuste aux formes de clusters et outliers, mais sensible aux paramètres)
        # Les paramètres eps et min_samples sont cruciaux et demandent une exploration
        dbscan_prod = DBSCAN(eps=0.5, min_samples=max(1, int(X6_prepared.shape[0]*0.1))) # Exemple de paramètres
        prod_features['cluster_dbscan'] = dbscan_prod.fit_predict(X6_prepared)
        
        # Le score de silhouette n'est pas toujours pertinent pour DBSCAN s'il y a des outliers (-1)
        # On ne calcule que si plus d'un cluster est trouvé (excluant les outliers)
        valid_clusters_dbscan = prod_features['cluster_dbscan'][prod_features['cluster_dbscan'] != -1]
        if len(np.unique(valid_clusters_dbscan)) > 1:
            silhouette_dbscan = silhouette_score(X6_prepared[prod_features['cluster_dbscan'] != -1], valid_clusters_dbscan)
            print(f"DBSCAN (Produits) Silhouette Score (sans outliers): {silhouette_dbscan:.2f}")
        print("Premiers produits avec leurs clusters DBSCAN:")
        print(prod_features[['id_produit', 'cluster_dbscan']].head())
    else:
        print("Pas assez de clusters potentiels pour la segmentation des produits.")
else:
    print("Pas assez de produits uniques pour la segmentation.")


# --- Tâche 7: Segmentation fournisseurs ---
print("\n--- Tâche 7: Segmentation fournisseurs ---")
df_task7 = df_global.copy()
# Agréger les données par fournisseur
fourn_features = df_task7.groupby('id_fournisseur').agg(
    fiabilité_moyenne=('fiabilité', 'mean'),
    delai_moyen_global=('délai_moyen_jours', 'mean'), # Déjà un délai moyen, on pourrait prendre la moyenne des delais de livraison effectifs
    ville_mode=('ville', lambda x: x.mode()[0] if not x.mode().empty else 'N/A'),
    pays_mode=('pays', lambda x: x.mode()[0] if not x.mode().empty else 'N/A'),
    taille_fournisseur=('taille_fournisseur_nb_produits', 'first') # Déjà calculé
).reset_index()

X7_cols_num = ['fiabilité_moyenne', 'delai_moyen_global', 'taille_fournisseur']
X7_cols_cat = ['ville_mode', 'pays_mode']
X7_processed = fourn_features.copy()

preprocessor7 = get_preprocessor(X7_cols_num, X7_cols_cat)
X7_prepared = preprocessor7.fit_transform(X7_processed[X7_cols_num + X7_cols_cat])

if X7_prepared.shape[0] > 1:
    n_clusters_fourn = min(3, X7_prepared.shape[0]) # Exemple: 3 clusters
    if n_clusters_fourn > 1:
        kmeans_fourn = KMeans(n_clusters=n_clusters_fourn, random_state=42, n_init='auto')
        fourn_features['cluster_kmeans'] = kmeans_fourn.fit_predict(X7_prepared)
        
        silhouette_kmeans_f = silhouette_score(X7_prepared, fourn_features['cluster_kmeans'])
        print(f"K-Means (Fournisseurs) Silhouette Score: {silhouette_kmeans_f:.2f}")
        print("Premiers fournisseurs avec leurs clusters K-Means:")
        print(fourn_features[['id_fournisseur', 'cluster_kmeans']].head())
    else:
        print("Pas assez de clusters potentiels pour la segmentation des fournisseurs.")
else:
    print("Pas assez de fournisseurs uniques pour la segmentation.")


# --- Tâche 8: Classification du fournisseur fiable ---
print("\n--- Tâche 8: Classification du fournisseur fiable ---")
df_task8 = df_global.copy().dropna(subset=['fiabilité_catégorie'])
# Agréger les données par fournisseur pour la classification
fourn_class_features = df_task8.groupby('id_fournisseur').agg(
    delai_moyen_liv=('délai_livraison_jours', 'mean'),
    montant_total_moyen=('montant_total', 'mean'),
    quantite_moyenne=('quantité', 'mean'),
    nb_ruptures=('rupture', 'sum'), # Nombre de fois où ce fournisseur a été associé à une rupture
    fiabilité_cible=('fiabilité_catégorie', 'first') # Cible
).reset_index()


X8_cols_num = ['delai_moyen_liv', 'montant_total_moyen', 'quantite_moyenne', 'nb_ruptures']
# Pas de X8_cols_cat dans cet exemple agrégé, mais on pourrait en ajouter
y8_col = 'fiabilité_cible'

X8 = fourn_class_features[X8_cols_num] # S'il y avait des X8_cols_cat, les ajouter ici
y8 = fourn_class_features[y8_col]

le8 = LabelEncoder()
y8_encoded = le8.fit_transform(y8)

if len(np.unique(y8_encoded)) > 1: # S'assurer qu'il y a plus d'une classe
    X8_train, X8_test, y8_train, y8_test = train_test_split(X8, y8_encoded, test_size=0.2, random_state=42, stratify=y8_encoded)
    
    # Prétraitement simplifié car que des numériques ici (à adapter si catégorielles)
    preprocessor8_num_only = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    model8 = RandomForestClassifier(random_state=42)
    pipeline8 = Pipeline(steps=[('preprocessor', preprocessor8_num_only), ('classifier', model8)])
    pipeline8.fit(X8_train, y8_train)
    y8_pred = pipeline8.predict(X8_test)

    accuracy8 = accuracy_score(y8_test, y8_pred)
    print(f"Résultats pour Random Forest Classifier (Fiabilité fournisseur):")
    print(f"  Accuracy: {accuracy8:.2f}")
    print("  Classification Report:")
    print(classification_report(y8_test, y8_pred, target_names=le8.classes_, zero_division=0))
else:
    print("Pas assez de classes différentes pour la classification de fiabilité fournisseur.")

print("\n--- Fin des Tâches ---")
Use code with caution.
Python

SyntaxError: invalid syntax (3884377133.py, line 203)